In [5]:
import pandas as pd

In [1]:
import geopandas as gpd

In [2]:
import altair as alt

In [6]:
drug_arrest = pd.read_csv('/Users/mengyu/code/sfpd_data/data/processed/drug_arrest_data.csv')
drug_arrest.groupby('Incident Day of Week').size().sort_values(ascending=False)

Incident Day of Week
Wednesday    3577
Tuesday      3054
Thursday     2879
Friday       2292
Monday       2100
Saturday     1868
Sunday       1668
dtype: int64

In [7]:
drug_arrest.groupby('Incident Category').size().sort_values(ascending=False)

Incident Category
Drug Offense      17195
Drug Violation      243
dtype: int64

In [8]:
drug_arrest.groupby(['Incident Category','Incident Description']).size().sort_values(ascending=False)

Incident Category  Incident Description                     
Drug Offense       Narcotics Paraphernalia, Possession of       4254
                   Methamphetamine, Possession For Sale         1936
                   Cocaine, Base/rock, Possession For Sale      1790
                   Methamphetamine Offense                      1546
                   Heroin, Possession For Sale                  1508
                                                                ... 
                   Barbiturates, Possession For Sale               1
                   Barbiturates, Transportation                    1
                   Methadone, Transportation                       1
                   Methadone, Sale                                 1
                   Controlled Substance, Presence Where Used       1
Length: 62, dtype: int64

In [9]:
drug_arrest.groupby('Analysis Neighborhood').size().sort_values(ascending=False).head(10)

Analysis Neighborhood
Tenderloin                        8613
South of Market                   2962
Mission                           1740
Financial District/South Beach     451
Bayview Hunters Point              437
Castro/Upper Market                291
Western Addition                   273
Haight Ashbury                     242
Nob Hill                           213
North Beach                        137
dtype: int64

In [10]:
drug_arrest.groupby(['Incident Day of Week','Incident Time']).size()

Incident Day of Week  Incident Time
Friday                00:00            20
                      00:01             3
                      00:02             2
                      00:03             1
                      00:07             1
                                       ..
Wednesday             23:50             2
                      23:52             2
                      23:55             2
                      23:56             3
                      23:59             2
Length: 5403, dtype: int64

In [11]:
drug_arrest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17438 entries, 0 to 17437
Data columns (total 35 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Unnamed: 0                                            17438 non-null  int64  
 1   Incident Datetime                                     17438 non-null  object 
 2   Incident Date                                         17438 non-null  object 
 3   Incident Time                                         17438 non-null  object 
 4   Incident Year                                         17438 non-null  int64  
 5   Incident Day of Week                                  17438 non-null  object 
 6   Report Datetime                                       17438 non-null  object 
 7   Row ID                                                17438 non-null  int64  
 8   Incident ID                                           17

In [12]:
drug_arrest= drug_arrest.rename(columns={"Incident Date":'incident_date','Row ID':'row_id',
                                         'Incident Day of Week':'day_of_week','Incident Year':'year'})
drug_arrest['incident_date'] = pd.to_datetime(drug_arrest['incident_date'])
counts = drug_arrest.groupby(drug_arrest['incident_date'].dt.strftime('%Y-%m')).size().reset_index(name='count')
counts['incident_date_2'] = counts['incident_date']
counts.head()

,incident_date,count,incident_date_2
0,2018-01,337,2018-01
1,2018-02,356,2018-02
2,2018-03,423,2018-03
3,2018-04,348,2018-04
4,2018-05,384,2018-05


In [13]:
base = alt.Chart(counts).mark_bar(size=9).encode(
    alt.X('incident_date:T',title='Date'),
    y='count',
    tooltip = [alt.Tooltip('incident_date_2',title = 'Date'),alt.Tooltip('count',title='Drug Arrests')]
).properties(
    width=600,
    height=400,
    title='Incidents by Year-Month'
)

base.interactive()

alt.Chart(...)

In [14]:
month_week = drug_arrest.groupby([drug_arrest['incident_date'].dt.strftime('%m'),drug_arrest['day_of_week']]).size().reset_index(name='count')
month_week.head()

,incident_date,day_of_week,count
0,01,Friday,253
1,01,Monday,201
2,01,Saturday,206
3,01,Sunday,206
4,01,Thursday,287


In [15]:
alt.Chart(month_week).mark_rect().encode(
    alt.X('incident_date:O',title = 'Month'),
    alt.Y('day_of_week',title = 'Day of Week'),
    color='count'
)

alt.Chart(...)

In [16]:
yr_month_week = drug_arrest.groupby([drug_arrest['year'],drug_arrest['incident_date'].dt.strftime('%m'),drug_arrest['day_of_week']]).size().reset_index(name='count')
yr_month_week.head()
alt.Chart(yr_month_week).mark_rect().encode(
    alt.X('incident_date:O',title = 'Month'),
    alt.Y('day_of_week',title = 'Day of Week'),
    color='sum(count)'
)

alt.Chart(...)

In [22]:
years =[2018,2019,2020,2021,2022,2023]
genre_dropdown = alt.binding_select(options=years, name="Year")
genre_select = alt.selection_point(fields=['year'], bind=genre_dropdown)

filter_years = base.add_params(
    genre_select
).transform_filter(
    genre_select
).properties(title="Dropdown Filtering")

AttributeError: module 'altair' has no attribute 'selection_point'

In [23]:
base = alt.Chart(counts).mark_bar(size=10).encode(
    x='incident_date',
    y='count',
).properties(
    width=600,
    height=400
).add_params(brush)
brush = alt.selection_interval()
upper = base.encode(
    alt.X('incident_date:T', scale=alt.Scale(domain=brush)),  
    tooltip = [alt.Tooltip('incident_date'),alt.Tooltip('count',title='Drug Arrests')]
).properties(title='Incidents by Year-Month')

# lower = base.properties(
#     height=60
# ).add_params(brush)

upper.interactive()

AttributeError: 'Chart' object has no attribute 'add_params'